In [ ]:
# !pip install langchain 
# !pip install gradio 
# !pip install -U langchain-community

In [ ]:
# !pip install chromadb

In [ ]:
# !pip install pymupdf

In [ ]:
import gradio as gr
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings

import warnings
warnings.filterwarnings('ignore')

In [ ]:
def process_pdf(pdf_bytes):
    if pdf_bytes is None:
        return None, None, None

    loader = PyMuPDFLoader(pdf_bytes)
    data = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500, chunk_overlap=100
    )
    chunks = text_splitter.split_documents(data)

    embeddings = OllamaEmbeddings(model="deepseek-r1")
    vectorstore = Chroma.from_documents(
        documents=chunks, embedding=embeddings, persist_directory="./chroma_db"
    )
    retriever = vectorstore.as_retriever()

    return text_splitter, vectorstore, retriever

In [ ]:
def combine_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
import re
from ollama import Client


def ollama_llm(question, context):
    
    oclient = Client(
        host='http://192.168.1.34:11434',
        headers={'x-some-header': 'some-value'}
    )

    formatted_prompt = f"Question: {question}\n\nContext: {context}"

    print('*** formatted_prompt_', formatted_prompt)

    response = oclient.chat(
        model="deepseek-r1:1.5b",
        messages=[{"role": "user", "content": formatted_prompt}],
    )

    response_content = response["message"]["content"]

    # Remove content between <think> and </think> tags to remove thinking output
    final_answer = re.sub(r"<think>.*?</think>", "",
                          response_content, flags=re.DOTALL).strip()

    return final_answer

In [ ]:
def rag_chain(question, text_splitter, vectorstore, retriever):
    retrieved_docs = retriever.invoke(question)
    print('*** retrieved_docs:', retrieved_docs)

    formatted_content = combine_docs(retrieved_docs)
    return ollama_llm(question, formatted_content)

In [ ]:
def ask_question(pdf_bytes, question):
    text_splitter, vectorstore, retriever = process_pdf(pdf_bytes)

    if text_splitter is None:
        return None  # No PDF uploaded
    
    print('*** text_splitter:', text_splitter)

    result = rag_chain(question, text_splitter, vectorstore, retriever)
    return {result}


interface = gr.Interface(
    fn=ask_question,
    inputs=[
        gr.File(label="Upload PDF (optional)"),
        gr.Textbox(label="Ask a question"),
    ],
    outputs="text",
    title="Ask questions about your PDF",
    description="Use DeepSeek-R1 to answer your questions about the uploaded PDF document.",
)

interface.launch()